In [1]:
import veloxchem as vlx

In [2]:
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [3]:
import adcc
from mpi4py import MPI
import sys
import numpy
numpy.set_printoptions(precision=4, suppress=True)

In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classi

In [4]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 2}

In [5]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [6]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [7]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [8]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.546314 a.u.         1.388593 Debye                                     
                                   Y   :        -0.000000 a.u.        -0.000000 Debye                                     
                                   Z   :         0.386302 a.u.         0.981881 Debye                                     
                                 Total :         0.669095 a.u.         1.700671 Debye                                     
                

In [9]:
# Linear Response
#lr_eig_drv = vlx.LinearResponseEigenSolver(comm, ostream)
#lr_eig_drv.update_settings(rsp_settings, method_settings)
#lr_results = lr_eig_drv.compute(molecule, basis, scfdrv.scf_tensors)
#lr_eig_drv.ostream.flush()

In [10]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 2                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [11]:
# Get MO coefficients, eigenvalues and eigenvectors

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vals = lr_results["eigenvalues"]
#eig_vecs = lr_results["eigenvectors"]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)
print("mo_occ:")
print(mo_occ)
print()
print("mo_vir:")
print(mo_vir)

mo_occ:
[[-0.9941 -0.2329  0.0001  0.1002 -0.    ]
 [-0.0268  0.8319 -0.0004 -0.5233  0.    ]
 [ 0.0061  0.1603  0.4415  0.269   0.    ]
 [ 0.0061  0.1601 -0.4416  0.2693 -0.    ]
 [ 0.      0.      0.     -0.     -1.    ]
 [-0.0025  0.0731  0.4902  0.4582 -0.    ]
 [-0.0035  0.1032 -0.3464  0.6484 -0.    ]]

mo_vir:
[[-0.1356 -0.0002]
 [ 0.9082  0.0017]
 [-0.8062 -0.8437]
 [-0.8082  0.8411]
 [ 0.      0.    ]
 [ 0.4112  0.8244]
 [ 0.5841 -0.5815]]


In [12]:
print(scfdrv.scf_tensors.keys())
# Transform the Fock matrix from AO to MO basis
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])
[[ 1.  0. -0. -0. -0. -0.  0.]
 [ 0.  1.  0.  0. -0. -0. -0.]
 [-0.  0.  1.  0. -0.  0. -0.]
 [-0.  0.  0.  1. -0. -0.  0.]
 [-0. -0. -0. -0.  1. -0. -0.]
 [-0. -0.  0. -0. -0.  1. -0.]
 [ 0. -0. -0.  0. -0. -0.  1.]]
[[-20.2334   0.      -0.       0.      -0.       0.       0.    ]
 [  0.      -1.2657   0.      -0.      -0.       0.       0.    ]
 [ -0.       0.      -0.6293   0.      -0.       0.      -0.    ]
 [  0.      -0.       0.      -0.4417   0.       0.       0.    ]
 [ -0.      -0.      -0.       0.      -0.3876   0.       0.    ]
 [  0.       0.       0.       0.       0.       0.6028   0.    ]
 [  0.       0.      -0.       0.       0.       0.       0.7659]]


In [13]:
# Convert the first eigenvector to matrix form
#eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(eig_vec, nocc, nocc + nvir)

In [14]:
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

In [15]:
print(tda_eig_vec)

[-0.    -0.     0.     0.    -0.     0.     0.    -0.     1.    -0.001]


In [16]:
#tda_eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(tda_eig_vec, nocc, nocc + nvir)

In [17]:
print(tda_eig_vec_as_mat)

[[-0.    -0.   ]
 [ 0.     0.   ]
 [-0.     0.   ]
 [ 0.    -0.   ]
 [ 1.    -0.001]]


In [18]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=3, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     6     0.0028135   17ms  [0.4834 0.5742 0.6052]
  2    10     3.115e-29   22ms  [0.4834 0.5742 0.6021]
=== Converged ===
    Number of matrix applies:    10
    Total solver time:           41.451ms


In [19]:
adc1_eig_vec = adc1.excitation_vector[0]["s"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy
adc1.state_dipole_moment

/home/emi/.local/lib/python3.6/site-packages/adcc/AmplitudeVector.py:78: UserWarning: Using the list interface of AmplitudeVector is deprecated and will be removed in version 0.16.0. Use block labels like 'ph', 'pphh' instead.
  warnings.warn("Using the list interface of AmplitudeVector is "


array([[-0.2108, -0.    , -0.1424],
       [ 0.0419,  0.    ,  0.0234],
       [-0.3315,  0.    , -0.2288]])

In [20]:
print(adc1_eig_vec)
print(adc1_eig_vals)
print(adc1.describe())

[[ 0.     -0.      0.      0.    ]
 [-0.      0.      0.      0.    ]
 [-0.     -0.      0.      0.    ]
 [ 0.      0.      0.      0.    ]
 [-0.7071  0.0007  0.      0.    ]
 [ 0.      0.      0.     -0.    ]
 [ 0.      0.     -0.      0.    ]
 [ 0.      0.     -0.     -0.    ]
 [ 0.      0.      0.      0.    ]
 [ 0.      0.     -0.7071  0.0007]]
[0.4834 0.5742 0.6021]
+----------------------------------------------------+
| adc1                          singlet ,  converged |
+----------------------------------------------------+
|  #        excitation energy     osc str    |v1|^2  |
|          (au)           (eV)                       |
|  0     0.4834361      13.15497   0.0038         1  |
|  1     0.5742004      15.62479   0.0000         1  |
|  2      0.602137      16.38498   0.0546         1  |
+----------------------------------------------------+



In [21]:
print(tda_eig_vecs[:,0])
print(tda_eig_vals)

[-0.    -0.     0.     0.    -0.     0.     0.    -0.     1.    -0.001]
[0.4834 0.5742]


In [22]:
numpy.linalg.norm(adc1_eig_vec)

1.0

In [23]:
0.9999995/numpy.sqrt(2.0)

0.7071064276331569

In [24]:
0.0009539/numpy.sqrt(2.0)

0.0006745091585738476

In [25]:
print(adc1_eig_vec.shape)
print(adc1_eig_vec)
print()
print(tda_eig_vec_as_mat.shape)
print(tda_eig_vec_as_mat/numpy.sqrt(2))

(10, 4)
[[ 0.     -0.      0.      0.    ]
 [-0.      0.      0.      0.    ]
 [-0.     -0.      0.      0.    ]
 [ 0.      0.      0.      0.    ]
 [-0.7071  0.0007  0.      0.    ]
 [ 0.      0.      0.     -0.    ]
 [ 0.      0.     -0.      0.    ]
 [ 0.      0.     -0.     -0.    ]
 [ 0.      0.      0.      0.    ]
 [ 0.      0.     -0.7071  0.0007]]

(5, 2)
[[-0.     -0.    ]
 [ 0.      0.    ]
 [-0.      0.    ]
 [ 0.     -0.    ]
 [ 0.7071 -0.0007]]


In [26]:
vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

[[-0.  0. -0.  0.  0.]
 [ 0. -0.  0. -0. -0.]
 [-0.  0. -0.  0.  0.]
 [ 0. -0.  0. -0. -0.]
 [ 0. -0.  0. -0. -1.]]


In [27]:
adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

#print(adcc_dm_oo)

foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

adcc_excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )

In [28]:
print(adcc_excitation_energy)

0.4834361086240134


$E = \frac{1}{4} \sum_{pqrs} \Gamma_{pqrs} \langle pq || rs \rangle$

$\langle ia || jb \rangle = - \langle ai || jb \rangle = - \langle ia || bj \rangle = \langle ai || bj \rangle$

analogous symmetry for $\boldsymbol{\Gamma}$

In [29]:
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*(
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        #+0.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, nvir:])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##bbbb
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, nvir:])##abba
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, nvir:])##baab
                        ))
print("vlx energy =", vlx_excitation_energy)

vlx energy = 0.4834361086240134


In [30]:
print(vlx_excitation_energy)

0.4834361086240134


In [31]:
adcc_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, :nvir, :nocc, :nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, nvir:, nocc:, :nvir])
    )
print(adcc_excitation_energy)

0.4834361086226625


In [32]:
print(ovov.shape)
#print(ovov)

(10, 4, 10, 4)


In [33]:
#print(vlx_DM_ovov)
#print()
print(vlx_DM_ovov.shape)
#print(adcc_DM_ovov)

(5, 2, 5, 2)


In [34]:
# Making the Vlx 2PDM twice the size with the other spin blocks
# and assign the corresponding values to it by comparing to the adcc matrix
# afterwards divide by two to yield the same numbers
##vlx_DM_ovov_full = numpy.zeros((2*nocc, 2*nvir, 2*nocc, 2*nvir))
#vlx_DM_ovov_full[:nocc, :nvir, :nocc, :nvir] = vlx_DM_ovov
##vlx_DM_ovov_full[0, 0, 0, 0] = vlx_DM_ovov_full[0, 3, 1, 0] = vlx_DM_ovov_full[1,0,0,3] = vlx_DM_ovov_full[1,3,1,3] = vlx_DM_ovov[0,0,0,0]
##vlx_DM_ovov_full[0, 0, 0, 2] = vlx_DM_ovov_full[0, 2, 0, 0] = vlx_DM_ovov_full[0,3,1,2] = vlx_DM_ovov_full[0,5,1,0] = vlx_DM_ovov[0,2,0,0]
##vlx_DM_ovov_full[1, 0, 0, 5] = vlx_DM_ovov_full[1, 2, 0, 3] = vlx_DM_ovov_full[1,3,1,5] = vlx_DM_ovov_full[1,5,1,3] = vlx_DM_ovov[0,2,0,0]
##vlx_DM_ovov_full[0, 2, 0, 2] = vlx_DM_ovov_full[0, 5, 1, 2] = vlx_DM_ovov_full[1,2,0,5] = vlx_DM_ovov_full[1,5,1,5] = vlx_DM_ovov[0,2,0,2]

### matrix elements were twice as large
##vlx_DM_ovov_full *= 0.5
# norm of the difference between the two density matrices does not seem to vanish entirely...
##print("norm of the difference of the 2PDMs = ", numpy.linalg.norm(adcc_DM_ovov - vlx_DM_ovov_full))
#print(vlx_DM_ovov_full)
##vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
##                        #(1.0*numpy.einsum('ij,ij->', adcc_dm_oo,foo)
##                        #+1.0*numpy.einsum('ij,ij->', adcc_dm_vv,fvv)
##                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
##                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov_full, ovov)
##                        )
##print("vlx exc. energy =", vlx_excitation_energy)
#print("adcc - vlx =", adcc_excitation_energy - vlx_excitation_energy)

In [35]:
print(adcc_DM_ovov.shape)
print()
#print(adcc_DM_ovov)
print()
#print(adcc_DM_ovov - vlx_DM_ovov_full) # here everything appears to be zero

(10, 4, 10, 4)




In [36]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
adcc_dm_oo_in_ao = 2.0*numpy.matmul(mo_occ, numpy.matmul(adcc_dm_oo[:nocc,:nocc], mo_occ.T))

vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
adcc_dm_vv_in_ao = 2.0*numpy.matmul(mo_vir, numpy.matmul(adcc_dm_vv[:nvir,:nvir], mo_vir.T))

vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao
adcc_dm_ao = adcc_dm_oo_in_ao + adcc_dm_vv_in_ao

print("VLX")
print(vlx_dm_ao)
print("\nADCC")
print(adcc_dm_ao)

contract_fock_mo = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo, foo[:nocc,:nocc])
                    +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir]))

contract_fock_ao = numpy.einsum('mn,mn->', vlx_dm_ao, fock)

print("contract_fock_ao =", contract_fock_ao)
print("contract_fock_mo =", contract_fock_mo)

VLX
[[ 0.0184 -0.1232  0.1092  0.1097 -0.     -0.0557 -0.0793]
 [-0.1232  0.8249 -0.7314 -0.7348  0.      0.3728  0.531 ]
 [ 0.1092 -0.7314  0.6486  0.6516 -0.     -0.3306 -0.4708]
 [ 0.1097 -0.7348  0.6516  0.6545  0.     -0.3321 -0.473 ]
 [-0.      0.     -0.      0.     -1.     -0.      0.    ]
 [-0.0557  0.3728 -0.3306 -0.3321 -0.      0.1685  0.24  ]
 [-0.0793  0.531  -0.4708 -0.473   0.      0.24    0.3418]]

ADCC
[[ 0.0184 -0.1232  0.1092  0.1097 -0.     -0.0557 -0.0793]
 [-0.1232  0.8249 -0.7314 -0.7348  0.      0.3728  0.531 ]
 [ 0.1092 -0.7314  0.6486  0.6516 -0.     -0.3306 -0.4708]
 [ 0.1097 -0.7348  0.6516  0.6545 -0.     -0.3321 -0.473 ]
 [-0.      0.     -0.     -0.     -1.     -0.     -0.    ]
 [-0.0557  0.3728 -0.3306 -0.3321 -0.      0.1685  0.24  ]
 [-0.0793  0.531  -0.4708 -0.473  -0.      0.24    0.3418]]
contract_fock_ao = 0.9904845983017319
contract_fock_mo = 0.9904845983017314


In [37]:
# Calculate the total density (including HF part)
# and calculated the unrelaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())

# electronic contribution
total_density = 2*scfdrv.scf_tensors['D'][0] + vlx_dm_ao
adcc_total_density = 2*scfdrv.scf_tensors['D'][0] + adcc_dm_ao
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])
adcc_electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * adcc_total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)
adcc_dipole_moment = (nuclear_dipole + adcc_electronic_dipole)

print("dipole moment [au] =", dipole_moment)
print()
print("adcc dm dipole moment [au] =", adcc_dipole_moment)
print()
print("ADC(1) dipole moment for water in Q-Chem:")
print("Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]")
print()
print("ADC(1) dipole moment from adcc:")
print(adc1.state_dipole_moment)

dipole moment [au] = [-0.2108 -0.     -0.1424]

adcc dm dipole moment [au] = [-0.2108 -0.     -0.1424]

ADC(1) dipole moment for water in Q-Chem:
Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]

ADC(1) dipole moment from adcc:
[[-0.2108 -0.     -0.1424]
 [ 0.0419  0.      0.0234]
 [-0.3315  0.     -0.2288]]


In [38]:
# Transform the excitation vectors from MO to AO basis
tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

[[ 0.     -0.      0.     -0.     -0.     -0.      0.    ]
 [ 0.     -0.     -0.      0.      0.      0.     -0.    ]
 [ 0.     -0.      0.      0.     -0.     -0.     -0.    ]
 [-0.      0.     -0.     -0.      0.      0.      0.    ]
 [ 0.1356 -0.9082  0.8054  0.809  -0.     -0.4105 -0.5846]
 [ 0.     -0.      0.      0.     -0.     -0.     -0.    ]
 [-0.      0.      0.     -0.     -0.     -0.      0.    ]]


In [39]:
#dir(scfdrv)

In [40]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)
#print("(pq|rs) =")
#print(pqrs)


\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

In [41]:
# Contraction of electron repulsion integrals in AO basis
# First reference value in MO
eri_contract_mo_adcc = numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
print("Contraction in MO adcc:", eri_contract_mo_adcc)

eri_contract_mo_vlx = (+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir]))##bbbb
print("Contraction in MO vlx: ", eri_contract_mo_vlx)

# Now contract everything in AO
eri_contract_ao = (0.5*numpy.einsum('mn,mn->', tda_eig_vec_ao, numpy.einsum('pt,mntp->mn', tda_eig_vec_ao, 4*pqrs)
                                   - numpy.einsum('pt,mptn->mn', tda_eig_vec_ao, 2*pqrs))
                    )
print("Contraction in AO:     ", eri_contract_ao)

Contraction in MO adcc: -0.5070484896790689
Contraction in MO vlx:  -0.507048489677718
Contraction in AO:      -0.5070484896777177


In [42]:
mo.shape
print(nocc + nvir)

7


## Orbital response, right hand side

### 1. Contributions from the 1PDMs
\begin{equation}
F^\gamma_{ia}=\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
=\sum_{\theta,\varphi,\sigma}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{1,\sigma}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
=\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{equation}

In [43]:
F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

[[-0.0183  0.    ]
 [-0.0827 -0.    ]
 [ 0.0005 -0.107 ]
 [-0.0908  0.0002]
 [ 0.     -0.    ]]


In [44]:
ooov = adc1.reference_state.eri("o1o1o1v1").to_ndarray()
ovvv = adc1.reference_state.eri("o1v1v1v1").to_ndarray()
adcc_1pdm_rhs = ( -numpy.einsum('jk,ikja->ia', adcc_dm_oo, ooov, optimize=True) 
                 -numpy.einsum('bc,icba->ia', adcc_dm_vv, ovvv, optimize=True)
                )
print(adcc_1pdm_rhs)

[[-0.0183  0.     -0.     -0.    ]
 [-0.0827 -0.     -0.     -0.    ]
 [ 0.0005 -0.107  -0.     -0.    ]
 [-0.0908  0.0002 -0.     -0.    ]
 [-0.      0.     -0.     -0.    ]
 [-0.     -0.     -0.0183  0.    ]
 [-0.     -0.     -0.0827 -0.    ]
 [-0.     -0.      0.0005 -0.107 ]
 [-0.     -0.     -0.0908  0.0002]
 [-0.     -0.     -0.      0.    ]]


\begin{equation}
F^\Gamma_{ia}=\sum_{\mu,\rho}C^\alpha_{\mu a}C^\alpha_{\rho i}\left[\sum_{\tau,\varphi}S_{\rho\tau}x^\alpha_{\tau\varphi}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\tau,\nu}S_{\rho\tau}x^\alpha_{\tau\nu}\sum_{\theta,\varphi,\sigma} x^\sigma_{\theta\varphi}(\mu\nu|\theta\varphi)\right]-\sum_{\mu,\rho}C^\alpha_{\rho i}C^\alpha_{\mu a}\left[\sum_{\tau,\varphi}S_{\mu\tau}x^\alpha_{\tau\varphi}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\rho\nu|\theta\varphi)-\sum_{\tau,\nu} S_{\mu\tau}x^\alpha_{\tau\nu}\sum_{\theta,\varphi,\sigma}x^\sigma_{\theta\varphi}(\rho\nu|\theta\varphi) \right]
\end{equation}

In [63]:
FG1_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG2_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG3_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG4_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )


FG1_vlx =( numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zp,mp->zm', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG1_vlx_ovlp =( numpy.einsum('rp,mp->rm',
                numpy.einsum('rz,zp->rp', ovlp, tda_eig_vec_ao),
                numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
            )

FG2_vlx =  numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zn,nm->zm', tda_eig_vec_ao,
                                    numpy.einsum('tp,mntp->nm',tda_eig_vec_ao,pqrs)
                                   )
                      )
#FG3_vlx =  numpy.einsum('mz,rz->rm', ovlp,
#                       numpy.einsum('zp,rp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('tn,rntp->rp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#FG4_vlx =  numpy.einsum('mz,rz->rm', ovlp,
#                       numpy.einsum('zn,rn->rz', tda_eig_vec_ao,
#                                    numpy.einsum('tp,rntp->rn',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
print("eigenvector in MO:")
print(tda_eig_vec_as_mat)
print()
print("eigenvector in AO:")
print(tda_eig_vec_ao)
print()
print("first term:")
print(FG1_try2)
print()
print("first term:")
print(FG1_vlx-FG2_vlx)
print()
F_2pdm_vlx_A = 0.5*FG1_try2 - FG2_try2 
F_2pdm_vlx_B =-0.5*FG3_try2 + FG4_try2

F_2pdm_vlx = F_2pdm_vlx_A + F_2pdm_vlx_B
#print(F_2pdm_vlx)
another_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_A))
                +numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_B))
               )
vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx)))
print(vlx_2pdm_rhs)
print()
print(another_vlx_2pdm_rhs)         

eigenvector in MO:
[[-0.    -0.   ]
 [ 0.     0.   ]
 [-0.     0.   ]
 [ 0.    -0.   ]
 [ 1.    -0.001]]

eigenvector in AO:
[[ 0.     -0.      0.     -0.     -0.     -0.      0.    ]
 [ 0.     -0.     -0.      0.      0.      0.     -0.    ]
 [ 0.     -0.      0.      0.     -0.     -0.     -0.    ]
 [-0.      0.     -0.     -0.      0.      0.      0.    ]
 [ 0.1356 -0.9082  0.8054  0.809  -0.     -0.4105 -0.5846]
 [ 0.     -0.      0.      0.     -0.     -0.     -0.    ]
 [-0.      0.      0.     -0.     -0.     -0.      0.    ]]

first term:
[[ 0.0001 -0.0014  0.0057  0.0057 -0.     -0.0028 -0.004 ]
 [-0.0017  0.0184 -0.0732 -0.0735  0.      0.0356  0.0508]
 [ 0.001  -0.0114  0.0453  0.0455 -0.     -0.022  -0.0314]
 [ 0.001  -0.0114  0.0455  0.0457 -0.     -0.0221 -0.0316]
 [-0.      0.     -0.     -0.      0.      0.      0.    ]
 [-0.0018  0.0197 -0.0787 -0.079   0.      0.0383  0.0546]
 [-0.0025  0.0281 -0.1122 -0.1126  0.      0.0546  0.0778]]

first term:
[[-0.  0.  0. -0.  0.

In [52]:
adcc_2pdm_rhs = (numpy.einsum('jakb,ijkb->ia', adcc_DM_ovov, ooov, optimize=True)
                + numpy.einsum('ibjc,jcab->ia', adcc_DM_ovov, ovvv, optimize=True)
                )
print(adcc_2pdm_rhs)

[[ 0.0194 -0.      0.      0.    ]
 [ 0.0193 -0.      0.      0.    ]
 [-0.0002  0.      0.      0.    ]
 [ 0.058  -0.0001  0.      0.    ]
 [ 0.      0.      0.      0.    ]
 [ 0.      0.      0.0194 -0.    ]
 [ 0.      0.      0.0193 -0.    ]
 [ 0.      0.     -0.0002  0.    ]
 [ 0.      0.      0.058  -0.0001]
 [ 0.      0.      0.      0.    ]]
